In [1]:
abs_path = "/home/zzz0054/bio/7-5"

In [39]:
for i in ["5008134", "5008135", "5008136", "5008137"]:
    !fastq-dump SRR{i}

Read 54157010 spots for SRR5008134
Written 54157010 spots for SRR5008134
Read 32548315 spots for SRR5008135
Written 32548315 spots for SRR5008135
Read 45918566 spots for SRR5008136
Written 45918566 spots for SRR5008136
Read 39039900 spots for SRR5008137
Written 39039900 spots for SRR5008137


In [7]:
!prepare_transcripts -g /home/zzz0054/bio/7-5/Saccharomyces_cerevisiae.R64-1-1.112.gtf -f /home/zzz0054/bio/7-5/Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa -o /home/zzz0054/bio/7-5/RiboCode_annot

[2024-07-09 04:28:18] Preparing annotation files ...
	Reading the GTF file: /home/zzz0054/bio/7-5/Saccharomyces_cerevisiae.R64-1-1.112.gtf .......
	Process the transcripts ....
	Saving the transcripts.pickle
[2024-07-09 04:28:19] The step of preparing transcript annotation has been completed.


In [8]:
!OutputTranscriptInfo -c /home/zzz0054/bio/7-5/RiboCode_annot/transcripts_cds.txt -g /home/zzz0054/bio/7-5/Saccharomyces_cerevisiae.R64-1-1.112.gtf -f /home/zzz0054/bio/7-5/RiboCode_annot/transcripts_sequence.fa -o /home/zzz0054/bio/7-5/longest.transcripts.info.txt -O /home/zzz0054/bio/7-5/all.transcripts.info.txt

Starting outputing longest trans...
6600  transcripts will be used in the follow analysis.

Finishing!
Starting outputing all trans...
Finishing!


In [9]:
!GetProteinCodingSequence -i /home/zzz0054/bio/7-5/RiboCode_annot/transcripts_sequence.fa  -c /home/zzz0054/bio/7-5/longest.transcripts.info.txt -o ProteinCodingSeq --mode whole --table 1

6600  transcripts will be used in the follow analysis.

Notes: There are 0 transcripts whose cds sequence cannot be divided by 3!
Finish the step of extracting sequences!


In [40]:
!mkdir -p {abs_path}/source_fastq
!mv SRR* /home/zzz0054/bio/7-5/source_fastq

In [45]:
!mkdir -p {abs_path}/trimmed_out/
for read in ["5008134", "5008135", "5008136", "5008137"]:
    !cutadapt -m 15 -M 35 --match-read-wildcards -a CTGTAGGCACCATCAAT -o {abs_path}/trimmed_out/SRR{read}.trimmed.fastq {abs_path}/source_fastq/SRR{read}.fastq > {abs_path}/trimmed_out/SRR{read}_trimmed.log

Done           00:08:30    54,157,010 reads @   9.4 µs/read;   6.36 M reads/minute
Done           00:05:01    32,548,315 reads @   9.2 µs/read;   6.49 M reads/minute
Done           00:06:50    45,918,566 reads @   8.9 µs/read;   6.71 M reads/minute
Done           00:05:56    39,039,900 reads @   9.1 µs/read;   6.58 M reads/minute


In [46]:
!mkdir -p  {abs_path}/quality_filtered
for read in ["5008134", "5008135", "5008136", "5008137"]:
    !fastq_quality_filter -Q33 -v -q 25 -p 75 -i {abs_path}/trimmed_out/SRR{read}.trimmed.fastq -o {abs_path}/quality_filtered/SRR{read}.trimmed.Qfilter.fastq > {abs_path}/quality_filtered/SRR{read}.Qfilter.log

In [52]:
!mkdir -p {abs_path}/bowtie_index/
!bowtie-build {abs_path}/Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa {abs_path}/bowtie_index/bowtie_index

Settings:
  Output files: "/home/zzz0054/bio/7-5/bowtie_index/bowtie_index.*.ebwt"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 5 (one in 32)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /home/zzz0054/bio/7-5/Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 3039276
Using parameters --bmax 2279457 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with

In [55]:
!mkdir -p  {abs_path}/aligned
for read in ["5008134", "5008135", "5008136", "5008137"]:
    !bowtie -n 0 -y -a --norc --best --strata -S -p 4 -l 15 --un={abs_path}/aligned/noncontam_SRR{read}.fastq {abs_path}/bowtie_index/bowtie_index -q {abs_path}/quality_filtered/SRR{read}.trimmed.Qfilter.fastq {abs_path}/aligned/SRR{read}.alin

Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 53438435
# reads with at least one alignment: 14453944 (27.05%)
# reads that failed to align: 38984491 (72.95%)
Reported 22960671 alignments
Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 32037352
# reads with at least one alignment: 8788039 (27.43%)
# reads that failed to align: 23249313 (72.57%)
Reported 18287809 alignments
Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 45282861
# reads with at least one alignment: 12887235 (28.46%)
# reads that failed to align: 32395626 (71.54%)
Reported 22053826 alignments
Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 38539844
# reads with at least one alignment: 9147596 

In [54]:
!STAR --runThreadN 8 \
--runMode genomeGenerate \
--genomeDir {abs_path}/STAR_genome_index/ \
--genomeFastaFiles {abs_path}/Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa \
--sjdbGTFfile {abs_path}/Saccharomyces_cerevisiae.R64-1-1.112.gtf \
--sjdbOverhang 100

	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --runThreadN 8 --runMode genomeGenerate --genomeDir /home/zzz0054/bio/7-5/STAR_genome_index/ --genomeFastaFiles /home/zzz0054/bio/7-5/Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa --sjdbGTFfile /home/zzz0054/bio/7-5/Saccharomyces_cerevisiae.R64-1-1.112.gtf --sjdbOverhang 100
	STAR version: 2.7.11b   compiled: 2024-03-19T08:38:59+0000 :/opt/conda/conda-bld/star_1710837244939/work/source
Jul 09 06:23:36 ..... started STAR run
Jul 09 06:23:36 ... starting to generate Genome files
Jul 09 06:23:37 ..... processing annotations GTF
!!!!! WARNING: --genomeSAindexNbases 14 is too large for the genome size=12157105, which may cause seg-fault at the mapping step. Re-run genome generation with recommended --genomeSAindexNbases 10
Jul 09 06:23:37 ... starting to sort Suffix Array. This may take a long time...
Jul 09 06:23:37 ... sorting Suffix Array chunks and saving them to disk...
Jul 09 06:23:39 ... loading chunks from disk, packing SA...
Jul

In [59]:
for read in ["5008134", "5008135", "5008136", "5008137"]:
    !STAR --runThreadN 8 --outFilterType Normal --outWigType wiggle --outWigStrand Stranded --outWigNorm RPM --alignEndsType EndToEnd --outFilterMismatchNmax 1 --outFilterMultimapNmax 1 --genomeDir {abs_path}/STAR_genome_index/ --readFilesIn {abs_path}/aligned/noncontam_SRR{read}.fastq --outFileNamePrefix  {abs_path}/star_out/{read}/ --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --outSAMattributes All

	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --runThreadN 8 --outFilterType Normal --outWigType wiggle --outWigStrand Stranded --outWigNorm RPM --alignEndsType EndToEnd --outFilterMismatchNmax 1 --outFilterMultimapNmax 1 --genomeDir /home/zzz0054/bio/7-5/STAR_genome_index/ --readFilesIn /home/zzz0054/bio/7-5/aligned/noncontam_SRR5008134.fastq --outFileNamePrefix /home/zzz0054/bio/7-5/star_out/5008134/ --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --outSAMattributes All
	STAR version: 2.7.11b   compiled: 2024-03-19T08:38:59+0000 :/opt/conda/conda-bld/star_1710837244939/work/source
Jul 09 06:46:05 ..... started STAR run
Jul 09 06:46:05 ..... loading genome
Jul 09 06:46:06 ..... started mapping
Jul 09 06:52:06 ..... finished mapping
Jul 09 06:52:07 ..... started sorting BAM
Jul 09 06:52:14 ..... started wiggle output
Jul 09 06:52:32 ..... finished successfully
	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --runThreadN 8 --outFilterType Normal --out

In [60]:
for read in ["5008134", "5008135", "5008136", "5008137"]:
    !samtools sort -T {abs_path}/star_out/{read}/Aligned.toTranscriptome.out.sorted -o {abs_path}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam {abs_path}/star_out/{read}/Aligned.toTranscriptome.out.bam
    !samtools index {abs_path}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam ## mapped to transcriptome
    !samtools index {abs_path}/star_out/{read}/Aligned.sortedByCoord.out.bam ## mapped to genome

[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[bam_sort_core] merging from 2 files and 1 in-memory blocks...
[bam_sort_core] merging from 3 files and 1 in-memory blocks...
[bam_sort_core] merging from 2 files and 1 in-memory blocks...


In [62]:
for read in ["5008134", "5008135", "5008136", "5008137"]:
    !mkdir -p {abs_path}/metaplots/{read}/
    !metaplots -a {abs_path}/RiboCode_annot -r {abs_path}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam -o {abs_path}/metaplots/{read}/

[2024-07-09 07:39:57] Create metaplot file and predict the P-site locations ...
	Loading transcripts.pickle ...
[2024-07-09 07:40:39] Complete prediction of the P-site locations
[2024-07-09 07:40:42] Create metaplot file and predict the P-site locations ...
	Loading transcripts.pickle ...
[2024-07-09 07:41:07] Complete prediction of the P-site locations
[2024-07-09 07:41:09] Create metaplot file and predict the P-site locations ...
	Loading transcripts.pickle ...
[2024-07-09 07:41:46] Complete prediction of the P-site locations
[2024-07-09 07:41:49] Create metaplot file and predict the P-site locations ...
	Loading transcripts.pickle ...
[2024-07-09 07:42:14] Complete prediction of the P-site locations


In [64]:
for read in ["5008134", "5008135", "5008136", "5008137"]:
    !mkdir -p {abs_path}/periodicity/{read}/
    !Periodicity -i {abs_path}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam -a {abs_path}/RiboCode_annot -o {abs_path}/periodicity/{read}/ -c {abs_path}/longest.transcripts.info.txt -L 25 -R 35

	Loading transcripts.pickle ...
6600  transcripts will be used in the follow analysis.

There are 6600 transcripts with both start and stop codon will be used for following analysis.
Start calculating read density...
There are 6600 used for statistics.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont:

In [74]:
!mkdir -p {abs_path}/DiffFrames/
!RiboDensityOfDiffFrames -f {abs_path}/attributes.txt -c {abs_path}/longest.transcripts.info.txt -o {abs_path}/DiffFrames/ --plot yes

your input : 4 bam files
6600  transcripts will be used in the follow analysis.

findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
Finish the step of plotting!
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
Finish the step of plotting!
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.


In [66]:
!mkdir {abs_path}/lengthDist
for read in ["5008134", "5008135", "5008136", "5008137"]:

    !LengthDistribution -i {abs_path}/quality_filtered/SRR{read}.trimmed.Qfilter.fastq -o {abs_path}/lengthDist/SRR{read} -f fastq

Start the step of length statistics...
The total reads number is: 53431688
The zero length number is: 0
The mean of reads length is: 26.875370903498315
The sd of reads length is: 3.4577385275217702
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
Finish the step of length statistics!
Start the step of length statistics...
The total reads number is: 32019060
The zero length number is: 0
The mean of reads length is: 25.78854760258421
The sd of reads length is: 4.564364516325098
findfont: Font family 'Arial' not f

In [68]:
!mkdir {abs_path}/DNAContam
for read in ["5008134", "5008135", "5008136", "5008137"]:
    !StatisticReadsOnDNAsContam -i  {abs_path}/star_out/{read}/Aligned.sortedByCoord.out.bam  -g {abs_path}/Saccharomyces_cerevisiae.R64-1-1.112.gtf  -o  {abs_path}/DNAContam/SRR{read}

mkdir: cannot create directory ‘/home/zzz0054/bio/7-5/DNAContam’: File exists
1000000 BAM alignments records processed.

2000000 BAM alignments records processed.

3000000 BAM alignments records processed.

4000000 BAM alignments records processed.

5000000 BAM alignments records processed.

6000000 BAM alignments records processed.

7000000 BAM alignments records processed.

8000000 BAM alignments records processed.

9000000 BAM alignments records processed.

10000000 BAM alignments records processed.

11000000 BAM alignments records processed.

12000000 BAM alignments records processed.

13000000 BAM alignments records processed.

14000000 BAM alignments records processed.

15000000 BAM alignments records processed.

16000000 BAM alignments records processed.

17000000 BAM alignments records processed.

18000000 BAM alignments records processed.

19000000 BAM alignments records processed.

20000000 BAM alignments records processed.

Start the step of plot...
findfont: Font family 'Ar

In [76]:
!mkdir -p {abs_path}/MetageneAnalysis/
!MetageneAnalysisForTheWholeRegions -f {abs_path}/attributes.txt -c {abs_path}/longest.transcripts.info.txt -o {abs_path}/MetageneAnalysis/ -b 15,90,15 -l 100 -n 10 -m 1 -e 5

6600  transcripts will be used in the follow analysis.

Lenght filter(-l)---Transcripts number filtered by criterion one is : 481
Lenght filter (3n)---Transcripts number filtered by criterion two is : 0
Total counts filter---Transcripts number filtered by criterion three is : 687
CDS density filter(RPKM-n or counts-n)---Transcripts number filtered by criterion four is : 885
CDS density filter(normed-m)---Transcripts number filtered by criterion five is : 0
Metaplots Transcript Number for bam file/home/zzz0054/bio/7-5/star_out/5008134/Aligned.toTranscriptome.out.sorted.bam is :4547
Finish the step of read density calculation!
Lenght filter(-l)---Transcripts number filtered by criterion one is : 481
Lenght filter (3n)---Transcripts number filtered by criterion two is : 0
Total counts filter---Transcripts number filtered by criterion three is : 776
CDS density filter(RPKM-n or counts-n)---Transcripts number filtered by criterion four is : 813
CDS density filter(normed-m)---Transcripts num

In [5]:
!PlotMetageneAnalysisForTheWholeRegions -i {abs_path}/MetageneAnalysis/_scaled_density_dataframe.txt -o {abs_path}/MetageneAnalysis/WholeRegions -g si-Ctrl,si-eIF5A -r si-Ctrl-1,si-Ctrl-2__si-eIF5A-1,si-eIF5A-2 -b 15,90,15 --mode all --xlabel-loc -0.4

your input file is: /home/zzz0054/bio/7-5/MetageneAnalysis/_scaled_density_dataframe.txt
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont:

In [3]:
## metagene analysis
!MetageneAnalysis -f {abs_path}/attributes.txt -c {abs_path}/longest.transcripts.info.txt -o {abs_path}/MetageneAnalysis/MetageneAnalysis > -U codon -M RPKM -u 0 -d 500 -l 100 -n 10 -m 1 -e 5 --norm yes -y 100 --CI 0.95 --type CDS
## plot
!PlotMetageneAnalysis -i {abs_path}/MetageneAnalysis/_scaled_density_dataframe.txt  -o {abs_path}/MetageneAnalysis/PlotMetageneAnalysis -u 0 -d 500 -g si-Ctrl,si-eIF5A -r si-Ctrl-1,si-Ctrl-2__si-eIF5A-1,si-eIF5A-2 -U codon --CI 0.95

your input : 4 bam files
6600  transcripts will be used in the follow analysis.

Lenght filter(-l)---Transcripts number filtered by criterion one is : 481
Lenght filter (3n)---Transcripts number filtered by criterion two is : 0
Total counts filter---Transcripts number filtered by criterion three is : 687
CDS density filter(RPKM-n or counts-n)---Transcripts number filtered by criterion four is : 885
CDS density filter(normed-m)---Transcripts number filtered by criterion five is : 0
Metaplots Transcript Number for bam file/home/zzz0054/bio/7-5/star_out/5008134/Aligned.toTranscriptome.out.sorted.bam is :4547
Finish the step of ribosomeDensityNormPerTrans
Lenght filter(-l)---Transcripts number filtered by criterion one is : 481
Lenght filter (3n)---Transcripts number filtered by criterion two is : 0
Total counts filter---Transcripts number filtered by criterion three is : 776
CDS density filter(RPKM-n or counts-n)---Transcripts number filtered by criterion four is : 813
CDS density filter(

In [4]:
## polarity calculation
!PolarityCalculation -f {abs_path}/attributes.txt -c {abs_path}/longest.transcripts.info.txt -o {abs_path}/MetageneAnalysis/polarity -n 64

## plot
!PlotPolarity -i {abs_path}/MetageneAnalysis/_scaled_density_dataframe.txt -o {abs_path}/MetageneAnalysis/polarity_plot -g si-Ctrl,si-eIF5A -r si-Ctrl-1,si-Ctrl-2__si-eIF5A-1,si-eIF5A-2  -y 5

your input : 4 bam files
6600  transcripts will be used in the follow analysis.

There are 6600 transcripts with both start and stop codon will be used for following analysis.
Metaplots Transcript Number for bam file/home/zzz0054/bio/7-5/star_out/5008134/Aligned.toTranscriptome.out.sorted.bam is :4785
The number of genes filtered by read counts equals to 64 is: 1815
There are 4785 genes used for plot polarity value
Metaplots Transcript Number for bam file/home/zzz0054/bio/7-5/star_out/5008135/Aligned.toTranscriptome.out.sorted.bam is :4385
The number of genes filtered by read counts equals to 64 is: 2215
There are 4385 genes used for plot polarity value
Metaplots Transcript Number for bam file/home/zzz0054/bio/7-5/star_out/5008136/Aligned.toTranscriptome.out.sorted.bam is :4726
The number of genes filtered by read counts equals to 64 is: 1874
There are 4726 genes used for plot polarity value
Metaplots Transcript Number for bam file/home/zzz0054/bio/7-5/star_out/5008137/Aligned.toTransc

In [6]:
# feature analysis
!mkdir {abs_path}/FeafureAnalysis
!RiboDensityForSpecificRegion -f {abs_path}/attributes.txt -c {abs_path}/longest.transcripts.info.txt -o {abs_path}/FeafureAnalysis/RiboDensityForSpecificRegion -U codon -M RPKM -L 1 -R 100


mkdir: cannot create directory ‘/home/zzz0054/bio/7-5/FeafureAnalysis’: File exists
your input : 4 bam files
6600  transcripts will be used in the follow analysis.

There are 6600 transcripts with both start and stop codon will be used for following analysis.
Finish the step of ribosomeDensityNormPerTrans
Finish the step of write_bam_file_density


In [ ]:
!RiboDensityAtEachKindAAOrCodon -f {abs_path}/attributes.txt -c {abs_path}/longest.transcripts.info.txt -o {abs_path}/FeafureAnalysis/RiboDensityAtEachKindAAOrCodon -M RPKM -S <select_trans.txt> -l 100 -n 10 --table 1 -F <longest_cds_sequence.fa>

In [8]:
# enrichment analysis
!mkdir {abs_path}/EnrichmentAnalysis
!RiboDensityAtEachPosition -c {abs_path}/longest.transcripts.info.txt -f {abs_path}/attributes.txt -o {abs_path}/EnrichmentAnalysis/RiboDensityAtEachPosition -U codon


mkdir: cannot create directory ‘/home/zzz0054/bio/7-5/EnrichmentAnalysis’: File exists
your input : 4 bam files
6600  transcripts will be used in the follow analysis.

There are 6600 transcripts with both start and stop codon will be used for following analysis.
Finish the step of ribosomeDensityAtEachPosition
Start calculating mean density...
Traceback (most recent call last):
  File "/home/zzz0054/miniconda3/envs/ribo4/bin/enrichmentMeanDensity", line 8, in <module>
    sys.exit(main())
  File "/home/zzz0054/miniconda3/envs/ribo4/lib/python3.8/site-packages/RiboMiner/enrichmentMeanDensity.py", line 65, in main
    meanDensity(inputFiles,options.outprefix)
  File "/home/zzz0054/miniconda3/envs/ribo4/lib/python3.8/site-packages/RiboMiner/enrichmentMeanDensity.py", line 46, in meanDensity
    ratio_dict=get_density_dict(f)
  File "/home/zzz0054/miniconda3/envs/ribo4/lib/python3.8/site-packages/RiboMiner/enrichmentMeanDensity.py", line 24, in get_density_dict
    with open(densityFile,'r

In [9]:

## use IP:MES1 for example
!enrichmentMeanDensity -i {abs_path}/EnrichmentAnalysis/RiboDensityAtEachPosition_si-eIF5A-1_cds_codon_coverage.txt,{abs_path}/EnrichmentAnalysis/RiboDensityAtEachPosition_si-eIF5A-2_cds_codon_coverage.txt -o {abs_path}/EnrichmentAnalysis/MES1_mean


Start calculating mean density...
Finish the step of mean density calculation!


In [ ]:

## all transcripts
!EnrichmentAnalysis --ctrl MES1_total_mean_density.txt --treat MES1_IP_mean_density.txt -c <longest.transcripts.info.txt> -o <output_prefix> -U codon -M RPKM -l 150 -n 10 -m 1 -e 30 --CI 0.95 -u 0 -d 500

## specific transcripts
!EnrichmentAnalysis --ctrl MES1_total_mean_density.txt --treat MES1_IP_mean_density.txt -c <longest.transcripts.info.txt> -o ARC1 -U codon -M RPKM -l 150 -n 10 -m 1 -e 30 --CI 0.95 -u 0 -d 500 -S ARC1.txt --id-type transcript-id

!PlotEnrichmentRatio -i ARC1_enrichment_dataframe.txt -o <output_prefix> -u 0 -d 500 --unit codon --mode all --slide-window y --axhline 1

!EnrichmentAnalysisForSingleTrans -i all_codon_ratio.txt.txt -s ARC1 -o ARC1 -c <longest.trans.info.txt>  --id-type gene_name --slide-window y --axhline 1